In [542]:
# import sys
# !pip install --prefix {sys.prefix} ipywidgets

In [543]:
from ipywidgets import ToggleButton, ToggleButtons, FloatSlider, IntSlider, Layout, Button, Box, AppLayout, IntProgress

#### Nagłówek: konfiguracja

In [544]:
confHeader = ToggleButton(
    description = 'Konfiguracja',
    disabled = False,
    button_style = 'danger', 
    layout = Layout(width = '72%', height = '50px')
)

#### Slidery: wielkość populacji, ilość bitów, liczba epok

In [545]:
populationSl = HBox([
    Label(
        'Liczba osobników populacji', 
        layout = Layout(width = '22%')), 
    IntSlider(
        min = 0, max = 200, step = 1, value = 20, 
        layout = Layout(width = '50%'),
    )
])

bitsSl = HBox([
    Label(
        'Liczba bitów w chromosomie', 
        layout = Layout(width = '22%')), 
    IntSlider(
        min = 0, max = 20, step = 1, value = 8, 
        layout = Layout(width = '50%'),
    )
])

epochSl = HBox([
    Label(
        'Liczba epok', 
        layout = Layout(width = '22%')), 
    IntSlider(
        min = 0, max = 500, step = 1, value = 100, 
        layout = Layout(width = '50%'),
    )
])

In [546]:
confSliders = Box(
    children = [populationSl, bitsSl, epochSl], 
    layout = Layout(display = 'flex', flex_flow = 'column', padding = "15px")
)
configLayout = AppLayout(header = confHeader, center = confSliders)

#### Nagłówek: operatory genetyczne

In [547]:
genHeader = ToggleButton(
    description = 'Operatory genetyczne',
    disabled = False,
    button_style = 'danger', 
    layout = Layout(width = '72%', height = '50px')
)

#### Przyciski: selekcja, krzyżowanie, mutacja

In [548]:
selectionBtn = ToggleButtons(
    options = ['najlepszych', 'kołem ruletki', 'turniejowa', 'strategia elitarna'],
    description = 'Selekcja: ',
    button_style = 'danger',
)

crossoverBtn = ToggleButtons(
    options = ['jednopunktowe', 'dwupunktowe', 'trzypunktowe', 'jednorodne'],
    description = 'Krzyżowanie: ',
    button_style = 'warning',
)

mutationBtn = ToggleButtons(
    options = ['jednopunktowa', 'dwupunktowa', 'brzegowa'],
    description = 'Mutacja: ',
    button_style = 'success',
)

#### Slidery: selekcja, krzyżowanie, mutacja

In [549]:
selectionSl = HBox([
    Label(
        'Procent osobników', 
        layout = Layout(width = '22%')), 
    IntSlider(
        min = 0, max = 100, step = 1, value = 30, 
        layout = Layout(width = '50%'), 
    )
])

crossoverSl = HBox([
    Label(
        'Prawdopodobieństwo krzyżowania', 
        layout = Layout(width = '22%')), 
    FloatSlider(
        min = 0, max = 1, step = 0.1, value = 0.9, 
        layout = Layout(width = '50%'),
    )
])

mutationSl = HBox([
    Label(
        'Prawdopodobieństwo mutacji', 
        layout = Layout(width = '22%')), 
    FloatSlider(
        min = 0, max = 1, step = 0.1, value = 0.2, 
        layout = Layout(width = '50%'),
    )
])

#### Layout: operatory genetyczne

In [550]:
buttons = Box(
    children = [selectionBtn, crossoverBtn, mutationBtn], 
    layout = Layout(display = 'flex', flex_flow = 'column')
)
geneticSliders = Box(
    children = [selectionSl, crossoverSl, mutationSl], 
    layout = Layout(display = 'flex', flex_flow = 'column', padding = "15px")
)
box = Box(
    children = [buttons, secondSliders], 
    layout = Layout(display = 'flex', flex_flow = 'column')
)

geneticLayout = AppLayout(header = genHeader, center = box)

In [551]:
startButton = ToggleButton(
    description = 'Rozpocznij',
    disabled = False,
    button_style = 'success', 
    layout = Layout(width = '72%', height = '50px')
)

In [552]:
app = Box(
    children = [configLayout, geneticLayout, startButton], 
    layout = Layout(display = 'flex', flex_flow = 'column')
)

In [553]:
app

Box(children=(AppLayout(children=(ToggleButton(value=False, button_style='danger', description='Konfiguracja',…

In [554]:
epochProgress = IntProgress(
    value = 7,
    min = 0,
    max = 10,
    step = 1,
    description = 'Epoka:',
    bar_style = '',
)

epochProgress

IntProgress(value=7, description='Epoka:', max=10)